In [3]:
#Utils.py contains all utility functions used during the inference process

import pandas as pd
import re
import nltk
import string
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score,classification_report,confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')
from wordcloud import WordCloud

import joblib
import importlib.util

DATA_DIR = r"C:\Users\K\Downloads\Great Learning\NLP\Project\Spam Detection Collection\spam.csv"
MODEL_DIR = r"C:\Users\K\Downloads\Great Learning\NLP\Project\spam_detection_model.pkl"


def module_from_file(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path) 
    module= importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

constants =  module_from_file("constants","/home/airflow/dags/Spam Detection Collection/constants.py")

def get_inference_data():
    encodings_to_try = ['utf-8', 'latin-1', 'ISO-8859-1', 'cp1252']

    for encoding in encodings_to_try:
        try:
            df = pd.read_csv(r"C:\Users\K\Downloads\Great Learning\NLP\Project\Spam Detection Collection\spam.csv", encoding=encoding)
            print(f"Successfully loaded using encoding: {encoding}")
            break
        except UnicodeDecodeError:
            print(f"Failed to load using encoding: {encoding}")
    df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)
    df = df.drop_duplicates(keep='first')
    df.rename(columns = {'v1':'label','v2':'text'},inplace=True)
    df.to_csv(f"{DATA_DIR}/spam_detect_inference.csv", index = False)

def preprocess_text(text):
    global stop_words
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    text = ' '.join(text.split())
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    text = ' '.join([word for word in tokens if word.lower() not in stop_words])
    df['text'] = df['text'].apply(preprocess_text)
    df.to_csv(f"{DATA_DIR}/spam_detect_preprocess_text.csv", index = False)

def predict_data():
    df = pd.read_csv(f"{DATA_DIR}/spam_detect_preprocess_text.csv")
    model = joblib.load(f"{MODEL_DIR}/spam_detect_model.joblib") 
    predictions = model.predict(df)
    prediction_df = pd.DataFrame (predictions)
    prediction_df.to_csv(f"{DATA_DIR}/spam_detect.csv", index = False)

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\home/airflow/dags/Spam Detection Collection/constants.py'